In [1]:
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
from IPython.display import Image
import shutil
from IPython.display import display
from PIL import Image

In [119]:
train_filenames = glob.glob('lamem/splits/train_*')

In [127]:
fn = train_filenames[2]
mem_scores = np.asarray([np.double(x.split(' ')[1][:-1]) for x in open(fn).readlines() for fn in train_filenames])
img_names = np.asarray([x.split(' ')[0] for x in open(fn).readlines() for fn in train_filenames])
img_names, img_names_where = np.unique(img_names, return_index=True)
mem_scores = mem_scores[img_names_where]
low_mem, hi_mem = 0.6, 0.65
valid = np.where(np.logical_and( mem_scores >= low_mem, mem_scores <= hi_mem))[0]
print(len(valid))
valid_images = img_names[valid]
valid_mem_scores = mem_scores[valid]
dirname = 'lamem/images/'


3684


Creating directory for version

In [139]:
mv v1 /Users/aidapiccato/PycharmProjects/concentration/concentration-game-mworks/

In [137]:
len(glob.glob('v1/*'))

3683

In [1]:
# for image in valid_images:    
#     try:
#         shutil.move(os.getcwd() +'/lamem/images/'+ image, 'v1')
#     except:
#         print(image)

Creating directory curr_images

In [17]:
# os.mkdir('curr_images')
images = glob.glob('images_v1/*')
np.random.shuffle(images)
curr_set = images[:300]
[shutil.copy(image, 'curr_images') for image in curr_set]

['curr_images/00020016.jpg',
 'curr_images/00016297.jpg',
 'curr_images/00030132.jpg',
 'curr_images/00045396.jpg',
 'curr_images/00043717.jpg',
 'curr_images/00008630.jpg',
 'curr_images/00028032.jpg',
 'curr_images/00044263.jpg',
 'curr_images/00043985.jpg',
 'curr_images/00050836.jpg',
 'curr_images/00046664.jpg',
 'curr_images/00023783.jpg',
 'curr_images/00038675.jpg',
 'curr_images/00036445.jpg',
 'curr_images/00009898.jpg',
 'curr_images/00040466.jpg',
 'curr_images/00020559.jpg',
 'curr_images/00040752.jpg',
 'curr_images/00031839.jpg',
 'curr_images/00037937.jpg',
 'curr_images/00008995.jpg',
 'curr_images/00038722.jpg',
 'curr_images/00022759.jpg',
 'curr_images/00050554.jpg',
 'curr_images/00057219.jpg',
 'curr_images/00050427.jpg',
 'curr_images/00035962.jpg',
 'curr_images/00025135.jpg',
 'curr_images/00022685.jpg',
 'curr_images/00058230.jpg',
 'curr_images/00025716.jpg',
 'curr_images/00015973.jpg',
 'curr_images/00012815.jpg',
 'curr_images/00002483.jpg',
 'curr_images/

Changing size of curr_images

In [25]:
images = glob.glob('curr_images/*')
desired_size = 368
for image in images:
    im = Image.open(image)
    old_size = im.size
#     print(old_size)
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = im.resize(new_size, Image.ANTIALIAS)
    # create a new image and paste the resized on it

    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))
    new_im.save(image)